### Librerie

In [8]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from PIL import Image
import cv2
import random

##  Configurazioni iniziali

In [9]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cpu


## Classe CLAHE per miglioramento immagini

In [10]:
class ApplyCLAHE:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        img_np = np.array(img)
        img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(img_lab)
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        l_clahe = clahe.apply(l)
        img_lab_clahe = cv2.merge((l_clahe, a, b))
        img_rgb_clahe = cv2.cvtColor(img_lab_clahe, cv2.COLOR_LAB2RGB)
        return Image.fromarray(img_rgb_clahe)

## Trasformazioni per immagini

In [11]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ApplyCLAHE(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

validation_image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ApplyCLAHE(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Dataset multimodale


In [12]:
class MultimodalDataset(Dataset):
    def __init__(self, df, tabular_data, labels, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.tabular_data = torch.tensor(tabular_data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.df.iloc[idx]['ID']
        image_path = os.path.join(self.image_dir, image_id + ".jpg")
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        if self.transform:
            image = self.transform(image)
        tabular = self.tabular_data[idx]
        label = self.labels[idx]
        return image, tabular, label

##  Modello multimodale: ResNet18 + Tabular NN

In [13]:
class OptimizedMultimodalClassifier(nn.Module):
    def __init__(self, tabular_input_dim):
        super().__init__()
        self.cnn = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.cnn.fc = nn.Identity()
        self.cnn_out_dim = 512
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(self.cnn_out_dim + 64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, image, tabular):
        image_feat = self.cnn(image)
        tabular_feat = self.tabular_net(tabular)
        combined = torch.cat([image_feat, tabular_feat], dim=1)
        return self.classifier(combined)

## Preprocessing del CSV e delle feature (DALL'SVC)

In [14]:
df = pd.read_csv("/content/mammografie.csv")
df = df.dropna(subset=['Severity', 'X', 'Y', 'Radius'])

df['Target'] = df['Severity'].map({'B': 0, 'M': 1})
df['X'] = pd.to_numeric(df['X'], errors='coerce')
df['Y'] = pd.to_numeric(df['Y'], errors='coerce')
df = df.dropna(subset=['X', 'Y'])

df['ID_Num'] = df['ID'].str.extract(r'(\d+)').astype(int)
df['Area'] = np.pi * (df['Radius'] ** 2)
df['DistanzaCentro'] = np.sqrt((df['X'] - 512)**2 + (df['Y'] - 512)**2)
df['Lato'] = df['ID_Num'].apply(lambda n: 'sinistro' if n % 2 == 1 else 'destro')
df['RadiusBin'] = pd.qcut(df['Radius'], q=5, labels=['XS', 'S', 'M', 'L', 'XL'])

def quadrante(row):
    if row['X'] < 512 and row['Y'] < 512: return 'Q0'
    elif row['X'] >= 512 and row['Y'] < 512: return 'Q1'
    elif row['X'] < 512 and row['Y'] >= 512: return 'Q2'
    else: return 'Q3'

df['Quadrante'] = df.apply(quadrante, axis=1)
df['Quadrante_Lato'] = df['Quadrante'] + '_' + df['Lato']

##  Encoding e standardizzazione feature tabellari

In [15]:
features = df[['Tissue', 'Class', 'Area', 'DistanzaCentro', 'Quadrante', 'RadiusBin']]
labels = df['Target'].values

numeric_features = ['Area', 'DistanzaCentro']
categorical_features = ['Tissue', 'Class', 'Quadrante', 'RadiusBin']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X_tabular = preprocessor.fit_transform(features)



## Creazione dataset e dataloader

In [16]:
dataset = MultimodalDataset(
    df, X_tabular, labels, "/content/Mammografie", transform=image_transform
)

train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
generator = torch.Generator().manual_seed(42)
train_ds, val_ds = random_split(dataset, [train_size, val_size], generator=generator)

train_ds.dataset.transform = image_transform
val_ds.dataset.transform = validation_image_transform

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

##  Inizializzazione modello e loss pesata

In [17]:
model = OptimizedMultimodalClassifier(tabular_input_dim=X_tabular.shape[1]).to(device)

class_counts = np.bincount(labels)
total_samples = len(labels)
class_weights = torch.tensor([
    total_samples / (2.0 * class_counts[0]),
    total_samples / (2.0 * class_counts[1])
], dtype=torch.float32).to(device)

print(f"Peso classi: {class_weights}")
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 144MB/s]


Peso classi: tensor([0.8750, 1.1667])


##  Training con early stopping

In [18]:
epochs = 15
best_val_acc = 0
patience = 7
patience_counter = 0

print("\nINIZIO...")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0
    for images, tabular, labels_batch in train_loader:
        images, tabular, labels_batch = images.to(device), tabular.to(device), labels_batch.to(device)
        outputs = model(images, tabular)
        loss = criterion(outputs, labels_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels_batch).sum().item()
        total_samples += labels_batch.size(0)

    train_acc = correct / total_samples
    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, tabular, labels_batch in val_loader:
            images, tabular, labels_batch = images.to(device), tabular.to(device), labels_batch.to(device)
            outputs = model(images, tabular)
            loss = criterion(outputs, labels_batch)
            val_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == labels_batch).sum().item()
            val_total += labels_batch.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())

    val_acc = val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1:2d} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        best_preds = all_preds.copy()
        best_labels = all_labels.copy()
        best_probs = all_probs.copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


INIZIO...
Epoch  1 | Train Loss: 0.7380 | Train Acc: 0.4458 | Val Loss: 0.6514 | Val Acc: 0.6389
Epoch  2 | Train Loss: 0.5117 | Train Acc: 0.8313 | Val Loss: 0.7021 | Val Acc: 0.5000
Epoch  3 | Train Loss: 0.4360 | Train Acc: 0.9639 | Val Loss: 0.7386 | Val Acc: 0.5000
Epoch  4 | Train Loss: 0.4086 | Train Acc: 0.9518 | Val Loss: 0.7128 | Val Acc: 0.5000
Epoch  5 | Train Loss: 0.3154 | Train Acc: 0.9880 | Val Loss: 0.7258 | Val Acc: 0.5278
Epoch  6 | Train Loss: 0.2247 | Train Acc: 0.9880 | Val Loss: 0.7033 | Val Acc: 0.5556
Epoch  7 | Train Loss: 0.2081 | Train Acc: 0.9880 | Val Loss: 0.7203 | Val Acc: 0.6111
Epoch  8 | Train Loss: 0.1382 | Train Acc: 0.9880 | Val Loss: 0.6891 | Val Acc: 0.6389
Early stopping at epoch 8


## 📈 Valutazione finale

In [19]:
print(f"\n=== Final Results ===")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print("\n=== Classification Report ===")
print(classification_report(best_labels, best_preds, target_names=["Benign", "Malignant"]))

if len(np.unique(best_labels)) > 1:
    auc_score = roc_auc_score(best_labels, best_probs)
    print(f"\nAUC Score: {auc_score:.4f}")

print("\n=== Confusion Matrix ===")
cm = confusion_matrix(best_labels, best_preds)
print(cm)



=== Final Results ===
Best Validation Accuracy: 0.6389

=== Classification Report ===
              precision    recall  f1-score   support

      Benign       0.73      0.44      0.55        18
   Malignant       0.60      0.83      0.70        18

    accuracy                           0.64        36
   macro avg       0.66      0.64      0.62        36
weighted avg       0.66      0.64      0.62        36


AUC Score: 0.7068

=== Confusion Matrix ===
[[ 8 10]
 [ 3 15]]
